In [1]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets

In [3]:
!pip install transformers

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [5]:
!ls /content/drive/MyDrive/DDI/pharma_bert_training/

ls: cannot access '/content/drive/MyDrive/DDI/pharma_bert_training/mod': No such file or directory


%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/DDI/pharma_bert_training/cleanup_no_redundancy_combined.txt",
    block_size=128,
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DDI/pharma_bert_training/model_parameters",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [6]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "/content/drive/MyDrive/DDI/pharma_bert_training/cleanup_no_redundancy_combined.txt"})

Using custom data configuration default-b1cbeefea4107e9f
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-b1cbeefea4107e9f/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
#block_size = tokenizer.model_max_length
block_size = 128

In [9]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
model_checkpoint = "lordtt13/COVID-SciBERT"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [12]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [13]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [14]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [15]:
from transformers import Trainer, TrainingArguments


In [23]:
training_args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    output_dir="/content/drive/MyDrive/DDI/pharma_bert_training/model_parameters_covid",
    overwrite_output_dir=True,
    num_train_epochs=2,
    save_steps=5000,
    save_total_limit=1,
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    #eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_covid/")